# Parte 1

Esta primeira parte do trabalho tem dois objetivos principais. O primeiro é verificar alguns parâmetros do hardware e do software utilizado e o segundo é que os estudantes preparem e se familiarizar com o ambiente do PostgreSQL.

## Tarefa 1 – Identificação do Sistema

Identifique o sistema que será usado para os experimentos, incluindo informações sobre o hardware e o Sistema Operacional utilizado. Sobre o tipo de processador, quantidade de memória RAM, tamanho do disco. Devem ser também apresentadas informações sobre as caches existentes. Sobre o Sistema Operacional, que devem ser Linux, incluir informações sobre qual a distribuição usada, versão do sistema, versão do Kernel, etc.

In [ ]:
print("=== INFORMAÇÕES DO SISTEMA OPERACIONAL ===")
!cat /etc/*release
!uname -r

print("\n=== INFORMAÇÕES DE CPU E CACHE ===")
!lscpu | grep -iE 'Model name|Socket|Thread|Core|cache|L[1-3]'

print("\n=== INFORMAÇÕES DE MEMÓRIA (RAM) ===")
!free -h

print("\n=== INFORMAÇÕES DE DISCO (Físico e Lógico) ===")
!lsblk -d -o name,model,size
print("-" * 20)
!df -h

In [ ]:
!pip install py-cpuinfo

In [5]:
# Source - https://stackoverflow.com/a/70810265
# Posted by 24_saurabh sharma
# Retrieved 2025-11-22, License - CC BY-SA 4.0

import psutil
import platform
from datetime import datetime
import cpuinfo
import socket
import uuid
import re


def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

def System_information():
    print("="*40, "System Information", "="*40)
    uname = platform.uname()
    print(f"System: {uname.system}")
    print(f"Node Name: {uname.node}")
    print(f"Release: {uname.release}")
    print(f"Version: {uname.version}")
    print(f"Machine: {uname.machine}")
    print(f"Processor: {uname.processor}")
    print(f"Processor: {cpuinfo.get_cpu_info()['brand_raw']}")
    print(f"Ip-Address: {socket.gethostbyname(socket.gethostname())}")
    print(f"Mac-Address: {':'.join(re.findall('..', '%012x' % uuid.getnode()))}")


    # Boot Time
    print("="*40, "Boot Time", "="*40)
    boot_time_timestamp = psutil.boot_time()
    bt = datetime.fromtimestamp(boot_time_timestamp)
    print(f"Boot Time: {bt.year}/{bt.month}/{bt.day} {bt.hour}:{bt.minute}:{bt.second}")


    # print CPU information
    print("="*40, "CPU Info", "="*40)
    # number of cores
    print("Physical cores:", psutil.cpu_count(logical=False))
    print("Total cores:", psutil.cpu_count(logical=True))
    # CPU frequencies
    cpufreq = psutil.cpu_freq()
    print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
    print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
    print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
    # CPU usage
    print("CPU Usage Per Core:")
    for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
        print(f"Core {i}: {percentage}%")
    print(f"Total CPU Usage: {psutil.cpu_percent()}%")


    # Memory Information
    print("="*40, "Memory Information", "="*40)
    # get the memory details
    svmem = psutil.virtual_memory()
    print(f"Total: {get_size(svmem.total)}")
    print(f"Available: {get_size(svmem.available)}")
    print(f"Used: {get_size(svmem.used)}")
    print(f"Percentage: {svmem.percent}%")



    print("="*20, "SWAP", "="*20)
    # get the swap memory details (if exists)
    swap = psutil.swap_memory()
    print(f"Total: {get_size(swap.total)}")
    print(f"Free: {get_size(swap.free)}")
    print(f"Used: {get_size(swap.used)}")
    print(f"Percentage: {swap.percent}%")



    # Disk Information
    print("="*40, "Disk Information", "="*40)
    print("Partitions and Usage:")
    # get all disk partitions
    partitions = psutil.disk_partitions()
    for partition in partitions:
        print(f"=== Device: {partition.device} ===")
        print(f"  Mountpoint: {partition.mountpoint}")
        print(f"  File system type: {partition.fstype}")
        try:
            partition_usage = psutil.disk_usage(partition.mountpoint)
        except PermissionError:
            # this can be catched due to the disk that
            # isn't ready
            continue
        print(f"  Total Size: {get_size(partition_usage.total)}")
        print(f"  Used: {get_size(partition_usage.used)}")
        print(f"  Free: {get_size(partition_usage.free)}")
        print(f"  Percentage: {partition_usage.percent}%")
    # get IO statistics since boot
    disk_io = psutil.disk_io_counters()
    print(f"Total read: {get_size(disk_io.read_bytes)}")
    print(f"Total write: {get_size(disk_io.write_bytes)}")

    ## Network information
    print("="*40, "Network Information", "="*40)
    ## get all network interfaces (virtual and physical)
    if_addrs = psutil.net_if_addrs()
    for interface_name, interface_addresses in if_addrs.items():
        for address in interface_addresses:
            print(f"=== Interface: {interface_name} ===")
            if str(address.family) == 'AddressFamily.AF_INET':
                print(f"  IP Address: {address.address}")
                print(f"  Netmask: {address.netmask}")
                print(f"  Broadcast IP: {address.broadcast}")
            elif str(address.family) == 'AddressFamily.AF_PACKET':
                print(f"  MAC Address: {address.address}")
                print(f"  Netmask: {address.netmask}")
                print(f"  Broadcast MAC: {address.broadcast}")
    ##get IO statistics since boot
    net_io = psutil.net_io_counters()
    print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
    print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")


if __name__ == "__main__":

    System_information()


======================================== System Information ========================================
System: Linux
Node Name: DESKTOP-7HG5CHA
Release: 6.6.87.2-microsoft-standard-WSL2
Version: #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025
Machine: x86_64
Processor: x86_64
Processor: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz
Ip-Address: 127.0.1.1
Mac-Address: 4d:0d:64:05:7a:e5
======================================== Boot Time ========================================
Boot Time: 2025/11/25 9:58:8
======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 0.00Mhz
Min Frequency: 0.00Mhz
Current Frequency: 2808.00Mhz
CPU Usage Per Core:
Core 0: 2.0%
Core 1: 2.9%
Core 2: 2.0%
Core 3: 1.0%
Core 4: 0.0%
Core 5: 1.0%
Core 6: 0.0%
Core 7: 2.0%
Total CPU Usage: 2.3%
======================================== Memory Information ========================================
Total: 7.69GB
Available: 5.56GB
Used: 2.14GB
Pe

## Tarefa 2 - Verificação de parâmetros de armazenamento

a) Identifique o modelo exato do disco utilizado nos experimentos (sem uso de sudo) e, a partir da documentação técnica do fabricante (datasheet ou site oficial), registre os seguintes parâmetros: 

- Número de superfícies (se disponível);
- Cilindros;
- Setores por trilha (se disponível);
- Velocidade de rotação;
- Latência rotacional;
- Tempos de seek:
    - Médio;
    - Máximo;
    - Mínimo.
- Tempo para a próxima trilha;
- Taxa de transferência. 

Apresente no notebook:

- O comando utilizado para obter o modelo do disco no Linux;
- O modelo encontrado.
- Os valores extraídos do datasheet (com a referência da fonte).

In [ ]:
print("=== COMANDO UTILIZADO ===")
print("!lsblk -d -o name,model,rota,size,type")
print("\n=== SAÍDA DO COMANDO ===")
# -d: evita mostrar partições
# -o: escolhe as colunas nome, modelo, rotação, tamanho
# rota: 1 = HDD, 0 = SSD
!lsblk -d -o name,model,rota,size,type

=== COMANDO UTILIZADO ===
!lsblk -d -o name,model,rota,size,type

=== SAÍDA DO COMANDO ===
NAME MODEL        ROTA   SIZE TYPE
sda  Virtual Disk    1 388.4M disk
sdb  Virtual Disk    1   186M disk
sdc  Virtual Disk    1     2G disk
sdd  Virtual Disk    1     1T disk


b) Utilizando o comando “stat” do Linux, verifique os parâmetros dos parâmetros de S.O. que serão utilizados para o disco.

In [10]:
print("=== DETALHES DO DIRETÓRIO ATUAL ===")
!stat .

print("\n=== DETALHES DO SISTEMA DE ARQUIVOS ===")
!stat -f .

=== DETALHES DO DIRETÓRIO ATUAL ===
  File: .
  Size: 4096      	Blocks: 0          IO Block: 4096   directory
Device: 44h/68d	Inode: 5348024558730164  Links: 1
Access: (0777/drwxrwxrwx)  Uid: ( 1000/     art)   Gid: ( 1000/     art)
Access: 2025-11-25 11:03:28.786871200 -0400
Modify: 2025-11-24 15:46:20.890348700 -0400
Change: 2025-11-24 15:46:20.890348700 -0400
 Birth: -



=== DETALHES DO SISTEMA DE ARQUIVOS ===
  File: "."
    ID: 100000000 Namelen: 255     Type: v9fs
Block size: 4096       Fundamental block size: 4096
Blocks: Total: 249887898  Free: 110699736  Available: 110699736
Inodes: Total: 999        Free: 1000000


c) Verifique o tamanho de bloco utilizado e mostre como alterar o tamanho dos blocos

In [12]:
print("=== 1. VERIFICAÇÃO DO TAMANHO DE BLOCO ATUAL ===")
# Obtém o dispositivo atual e pede ao blockdev o tamanho do bloco (Block Size)
!blockdev --getbsz $(df . --output=source | sed 1d)

=== 1. VERIFICAÇÃO DO TAMANHO DE BLOCO ATUAL ===
blockdev: cannot open C:\: No such file or directory


Para alterar o tamanho do bloco, é necessário reformatar a partição.

O comando abaixo exemplifica a formatação de uma partição definindo um tamanho de bloco de 4096 bytes:

``$ sudo mkfs.ext4 -b 4096 /dev/sdb1``

Contudo, este comando apagaria todos os dados da partição alvo.

Referências para fazer essa questão:

- https://unix.stackexchange.com/questions/145241/how-to-set-block-size-using-blockdev-command
- https://askubuntu.com/questions/1372387/resize-blocks-of-linux-filesystem
- https://stackoverflow.com/questions/67088392/how-to-change-block-size-on-xfs

## Tarefa 3 – Geração de um BD para testes

  Essa tarefa consiste na criação de banco de dados e povoamento destas tabelas com dados sintéticos. Para a definição do esquema das tabelas e os dados a serem carregados usaremos a especificação e os utilitários fornecidos pelos [Benchmark TPC-H](http://www.tpc.org/tpch/).
  
  Para geração do BD no PostgreSQL, siga as instruções disponíveis em https://github.com/foliveirafilho/tpch-pgsql.

### Comandos de setup de ambiente

In [ ]:
!sudo apt update

In [ ]:
!sudo apt install -y python3 python3-pip build-essential unzip postgresql postgresql-contrib libpq-dev

In [ ]:
!pip install jupyter ipython pandas psycopg2 sqlalchemy matplotlib

In [ ]:
!python3-venv
!source venv/bin/activate

In [ ]:
!service postgresql start

In [ ]:
# Comando pra configurar usuário do postgree
!sudo -u postgres psql

In [ ]:
CREATE USER icomp WITH ENCRYPTED PASSWORD 'icomp123';
CREATE DATABASE icomp;
GRANT ALL PRIVILEGES ON DATABASE icomp TO icomp;
ALTER DATABASE icomp OWNER TO icomp;
\q

### Construindo o tpch-dbgen


In [ ]:
!git clone https://github.com/foliveirafilho/tpch-pgsql.git
%cd tpch-pgsql

!wget -q https://github.com/electrum/tpch-dbgen/archive/32f1c1b92d1664dba542e927d23d86ffa57aa253.zip -O tpch-dbgen.zip
!unzip -q tpch-dbgen.zip && mv tpch-dbgen-32f1c1b92d1664dba542e927d23d86ffa57aa253 tpch-dbgen && rm tpch-dbgen.zip

!pip3 install -r requirements.txt

In [ ]:
# Prepara os dados
!python3 tpch_pgsql.py --host localhost --username icomp --password icomp123 --dbname icomp --dbgen-dir ./tpch-dbgen --scale 1 prepare

In [ ]:
# Carrega os dados
!python3 tpch_pgsql.py --host localhost --username icomp --password icomp123 --dbname icomp --dbgen-dir ./tpch-dbgen --scale 1 load

In [ ]:
# Comando de teste
!PGPASSWORD=icomp123 psql -h localhost -U icomp -d icomp -c "SELECT count(*) FROM lineitem;"

## Tarefa 4 – Execução de consultas

A segunda tarefa deste trabalho consiste em executar e analisar um conjunto de consultas analíticas do benchmark TPC-H sobre o banco de dados gerado na Tarefa 1, observando o comportamento do PostgreSQL em diferentes tipos de operações (filtros, junções, agregações e ordenações). O objetivo é compreender como o otimizador escolhe planos de execução e medir o tempo de resposta de consultas típicas de um sistema de apoio à decisão.

O TPC-H define 22 consultas padronizadas (Q1 a Q22), mas para este trabalho será utilizado um subconjunto representativo composto por 8 consultas que abrangem diferentes padrões de acesso.

Consultas a executar: Q1, Q3, Q5, Q6, Q7, Q9, Q10 e Q12. As consultas originais podem ser obtidos no repositório oficial do benchmark. Essas consultas devem ser executadas no mesmo banco TPC-H criado na Tarefa 1, utilizando o PostgreSQL.

Para cada consulta:

- Execute o comando SQL completo no PostgreSQL.
- Mostre as 10 primeiras linhas do resultado (ou um trecho representativo).
- Execute o comando EXPLAIN ANALYZE antes da consulta para exibir o plano de execução e o tempo real medido O comando EXPLAIN ANALYZE mostra as etapas internas do plano escolhido pelo otimizador, o número de linhas processadas e o tempo de cada operação.
- Registre o tempo total de execução e comente brevemente o tipo de acesso observado (por exemplo, varredura sequencial, uso de índice, junções em hash, etc.).

### Consultas SQL

In [36]:
sql_q1 = """
select
    l_returnflag,
    l_linestatus,
    sum(l_quantity) as sum_qty,
    sum(l_extendedprice) as sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
    avg(l_quantity) as avg_qty,
    avg(l_extendedprice) as avg_price,
    avg(l_discount) as avg_disc,
    count(*) as count_order
from
    lineitem
where
    l_shipdate <= date('1998-12-01') - interval '90' day
group by
    l_returnflag,
    l_linestatus
order by
    l_returnflag,
    l_linestatus;
"""

sql_q3 = """
select
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) as revenue,
    o_orderdate,
    o_shippriority
from
    customer,
    orders,
    lineitem
where
    c_mktsegment = 'BUILDING'
    and c_custkey = o_custkey
    and l_orderkey = o_orderkey
    and o_orderdate < date('1995-03-15')
    and l_shipdate > date('1995-03-15')
group by
    l_orderkey,
    o_orderdate,
    o_shippriority
order by
    revenue desc,
    o_orderdate
limit 10;
"""

sql_q5 = """
select
    n_name,
    sum(l_extendedprice * (1 - l_discount)) as revenue
from
    customer,
    orders,
    lineitem,
    supplier,
    nation,
    region
where
    c_custkey = o_custkey
    and l_orderkey = o_orderkey
    and l_suppkey = s_suppkey
    and c_nationkey = s_nationkey
    and s_nationkey = n_nationkey
    and n_regionkey = r_regionkey
    and r_name = 'ASIA'
    and o_orderdate >= date('1994-01-01')
    and o_orderdate < date('1994-01-01') + interval '1' year
group by
    n_name
order by
    revenue desc;
"""

sql_q6 = """
select
    sum(l_extendedprice * l_discount) as revenue
from
    lineitem
where
    l_shipdate >= date('1994-01-01')
    and l_shipdate < date('1994-01-01') + interval '1' year
    and l_discount between 0.06 - 0.01 and 0.06 + 0.01
    and l_quantity < 24;
"""

sql_q7 = """
select
    supp_nation,
    cust_nation,
    l_year,
    sum(volume) as revenue
from
    (
        select
            n1.n_name as supp_nation,
            n2.n_name as cust_nation,
            extract(year from l_shipdate) as l_year,
            l_extendedprice * (1 - l_discount) as volume
        from
            supplier,
            lineitem,
            orders,
            customer,
            nation n1,
            nation n2
        where
            s_suppkey = l_suppkey
            and o_orderkey = l_orderkey
            and c_custkey = o_custkey
            and s_nationkey = n1.n_nationkey
            and c_nationkey = n2.n_nationkey
            and (
                (n1.n_name = 'FRANCE' and n2.n_name = 'GERMANY')
                or (n1.n_name = 'GERMANY' and n2.n_name = 'FRANCE')
            )
            and l_shipdate between date('1995-01-01') and date('1996-12-31')
    ) as shipping
group by
    supp_nation,
    cust_nation,
    l_year
order by
    supp_nation,
    cust_nation,
    l_year;
"""

sql_q9 = """
select
    nation,
    o_year,
    sum(amount) as sum_profit
from
    (
        select
            n_name as nation,
            extract(year from o_orderdate) as o_year,
            l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity as amount
        from
            part,
            supplier,
            lineitem,
            partsupp,
            orders,
            nation
        where
            s_suppkey = l_suppkey
            and ps_suppkey = l_suppkey
            and ps_partkey = l_partkey
            and p_partkey = l_partkey
            and o_orderkey = l_orderkey
            and s_nationkey = n_nationkey
            and p_name like '%green%'
    ) as profit
group by
    nation,
    o_year
order by
    nation,
    o_year desc;
"""

sql_q10 = """
select
    c_custkey,
    c_name,
    sum(l_extendedprice * (1 - l_discount)) as revenue,
    c_acctbal,
    n_name,
    c_address,
    c_phone,
    c_comment
from
    customer,
    orders,
    lineitem,
    nation
where
    c_custkey = o_custkey
    and l_orderkey = o_orderkey
    and o_orderdate >= date('1993-10-01')
    and o_orderdate < date('1993-10-01') + interval '3' month
    and l_returnflag = 'R'
    and c_nationkey = n_nationkey
group by
    c_custkey,
    c_name,
    c_acctbal,
    c_phone,
    n_name,
    c_address,
    c_comment
order by
    revenue desc
limit 20;
"""

sql_q12 = """
select
    l_shipmode,
    sum(case
        when o_orderpriority = '1-URGENT'
            or o_orderpriority = '2-HIGH'
            then 1
        else 0
    end) as high_line_count,
    sum(case
        when o_orderpriority <> '1-URGENT'
            and o_orderpriority <> '2-HIGH'
            then 1
        else 0
    end) as low_line_count
from
    orders,
    lineitem
where
    o_orderkey = l_orderkey
    and l_shipmode in ('MAIL', 'SHIP')
    and l_commitdate < l_receiptdate
    and l_shipdate < l_commitdate
    and l_receiptdate >= date('1994-01-01')
    and l_receiptdate < date('1994-01-01') + interval '1' year
group by
    l_shipmode
order by
    l_shipmode;
"""

### Função de consulta

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import psycopg2

In [ ]:
def consultas_tarefa_4(query, limit=10, analyze=True):

    try:
        conn = psycopg2.connect(host="localhost", database="icomp", user="icomp", password="icomp123")
        cursor = conn.cursor()
        
        # --- PASSO 1: EXECUTAR E EXIBIR O EXPLAIN ANALYZE ---
        if analyze:
            
            print("=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===")
            explain_query = f"EXPLAIN ANALYZE {query}"
            cursor.execute(explain_query)
            
            plan_rows = cursor.fetchall()
            
            for row in plan_rows:
                print(row[0])
                
            print("\n" + "="*50 + "\n")

        # --- PASSO 2: EXECUTAR A CONSULTA PARA OS DADOS ---
        cursor.execute(query)
        
        colunas = [desc[0] for desc in cursor.description]
        
        data = cursor.fetchmany(limit)
        
        cursor.close()
        conn.close()

        print(f"=== RESULTADO DA CONSULTA ({limit} PRIMEIRAS LINHAS) ===")
        df = pd.DataFrame(data, columns=colunas)
        
        return df

    except Exception as e:
        print(f"Erro: {e}")
        return None

### Consultas e análises

#### Dicionário importante:

Partial HashAggregate: Quando tem algum paralelismo (diversos workers), um worker recebe uma carga de trabalho e repassa apenas um resumo dos seus dados. Por exemplo, ele recebe uma tabela de 500 pessoas, e devolve apenas o total de salário das 500 pessoas.

Gather merge: Momento em que o processo principal recolhe resultados feitos por processos em paralelo (diversos workers) e os junta de uma forma que mantenha o resultado ordenado.

Gather: Processo principal recolhe os resultados dos procesos paralelos e apenas amontoa, sem tentar manter ordenação no resultado funal.

Finalize GroupAggregate: Pega os resultados parciais e processa eles. Por exemplo, cada worker acha um valor, o processo principal os junta e retorna um valor.

#### Query 1

In [28]:
df_q1 = consultas_tarefa_4(sql_q1)
df_q1

=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===
Finalize GroupAggregate  (cost=231133.88..231135.84 rows=6 width=236) (actual time=3125.279..3131.455 rows=4 loops=1)
  Group Key: l_returnflag, l_linestatus
  ->  Gather Merge  (cost=231133.88..231135.28 rows=12 width=236) (actual time=3125.104..3131.224 rows=12 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Sort  (cost=230133.86..230133.87 rows=6 width=236) (actual time=3100.206..3100.208 rows=4 loops=3)
              Sort Key: l_returnflag, l_linestatus
              Sort Method: quicksort  Memory: 27kB
              Worker 0:  Sort Method: quicksort  Memory: 27kB
              Worker 1:  Sort Method: quicksort  Memory: 27kB
              ->  Partial HashAggregate  (cost=230133.65..230133.78 rows=6 width=236) (actual time=3100.147..3100.156 rows=4 loops=3)
                    Group Key: l_returnflag, l_linestatus
                    Batches: 1  Memory Usage: 32kB
                    Worker 0:  Batches: 

,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,37734107,56586554400.73,53758257134.8700,55909065222.827692,25.5220058532573370,38273.129734621672,0.04998529583839761162,1478493
1,N,F,991417,1487504710.38,1413082168.0541,1469649223.194375,25.5164719205229835,38284.467760848304,0.05009342667421629691,38854
2,N,O,74476040,111701729697.74,106118230307.6056,110367043872.497010,25.5022267695849915,38249.117988908270,0.04999658605370408037,2920374
3,R,F,37719753,56568041380.90,53741292684.6040,55889619119.831932,25.5057936126907707,38250.854626099657,0.05000940583012705647,1478870


##### Análise da Consulta Q1

**1. Tempo Total de Execução**:

  * ``3173.664 ms``.

**2. Estratégia de Scans**:

  * **Tabela Principal**: `lineitem`.
  * **Tipo de Acesso**: `Parallel Seq Scan`.
  * **Uso de Índice**? Não.

**3. Estratégia de Joins**:

  * Não há junções nesta consulta, apenas uma tabela.

**4. Comentário da Análise**:

Estratégia de Execução: O PostgreSQL optou por uma estratégia de Parallel Seq Scan, dividindo a leitura da tabela lineitem entre 2 workers e 1 líder para acelerar o processo.

Agregação em Duas Etapas: Para otimizar o tráfego de dados entre os processos, o banco utilizou Partial HashAggregate seguido de Finalize GroupAggregate.

Ordenação: O nó Sort foi executado individualmente por cada worker para permitir o uso do Gather Merge, que combina os resultados paralelos mantendo a ordenação solicitada na consulta, evitando uma reordenação custosa de todo o conjunto de dados final.

#### Consulta 3

In [29]:
df_q3 = consultas_tarefa_4(sql_q3)
df_q3

=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===
Limit  (cost=191564.17..191564.19 rows=10 width=44) (actual time=951.214..969.822 rows=10 loops=1)
  ->  Sort  (cost=191564.17..192339.79 rows=310250 width=44) (actual time=927.681..946.268 rows=10 loops=1)
        Sort Key: (sum((lineitem.l_extendedprice * ('1'::numeric - lineitem.l_discount)))) DESC, orders.o_orderdate
        Sort Method: top-N heapsort  Memory: 26kB
        ->  Finalize GroupAggregate  (cost=144029.58..184859.78 rows=310250 width=44) (actual time=908.170..943.423 rows=11620 loops=1)
              Group Key: lineitem.l_orderkey, orders.o_orderdate, orders.o_shippriority
              ->  Gather Merge  (cost=144029.58..177749.88 rows=258542 width=44) (actual time=908.149..936.134 rows=11620 loops=1)
                    Workers Planned: 2
                    Workers Launched: 2
                    ->  Partial GroupAggregate  (cost=143029.56..146907.69 rows=129271 width=44) (actual time=883.438..890.132 rows=3873 loop

,l_orderkey,revenue,o_orderdate,o_shippriority
0,2456423,406181.0111,1995-03-05,0
1,3459808,405838.6989,1995-03-04,0
2,492164,390324.0610,1995-02-19,0
3,1188320,384537.9359,1995-03-09,0
4,2435712,378673.0558,1995-02-26,0
5,4878020,378376.7952,1995-03-12,0
6,5521732,375153.9215,1995-03-13,0
7,2628192,373133.3094,1995-02-22,0
8,993600,371407.4595,1995-03-05,0
9,2300070,367371.1452,1995-03-13,0


##### Análise da Consulta Q3

**1. Tempo Total de Execução:**

- `993.332 ms`

**2. Estratégia de Acesso (Scans):**

**Tabelas e Acessos**:

- Tabela ``orders``: Parallel Seq Scan.
- Tabela ``customer``: Parallel Seq Scan.
- Tabela ``lineitem``: Index Scan, usando o índice idx_lineitem_orderkey.

**3. Estratégia de Joins:**

- **Parallel Hash Join**: Usado para juntar ``customer`` e ``orders``.
- **Nested Loop**: Usado para juntar o resultado anterior com ``lineitem``.

**4. Comentário da Análise:**

Primeiro, ele filtrou as tabelas customer e orders em paralelo (Parallel Seq Scan) e as uniu usando um Hash Join (eficiente para grandes volumes não ordenados).

Para a tabela lineitem (que é a maior de todas), em vez de lê-la inteira, o banco usou um Nested Loop combinado com um Index Scan. Isso significa que, para cada pedido filtrado nas etapas anteriores, ele buscou pontualmente os itens no índice, economizando muito tempo de I/O.

Por fim, a agregação foi feita em dois estágios partial e finalize e a ordenação final usou **Top-N heapsort** para retornar apenas as 10 linhas solicitadas sem precisar ordenar todo o conjunto de resultados na memória.

#### Consulta 5

In [41]:
df_q5 = consultas_tarefa_4(sql_q5)
df_q5

=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===
Sort  (cost=83509.85..83509.91 rows=25 width=136) (actual time=607.569..614.706 rows=5 loops=1)
  Sort Key: (sum((lineitem.l_extendedprice * ('1'::numeric - lineitem.l_discount)))) DESC
  Sort Method: quicksort  Memory: 25kB
  ->  Finalize GroupAggregate  (cost=83464.75..83509.27 rows=25 width=136) (actual time=606.610..614.670 rows=5 loops=1)
        Group Key: nation.n_name
        ->  Gather Merge  (cost=83464.75..83508.58 rows=50 width=136) (actual time=606.363..614.647 rows=15 loops=1)
              Workers Planned: 2
              Workers Launched: 2
              ->  Partial GroupAggregate  (cost=82464.73..82502.79 rows=25 width=136) (actual time=601.105..601.988 rows=5 loops=3)
                    Group Key: nation.n_name
                    ->  Sort  (cost=82464.73..82472.28 rows=3020 width=116) (actual time=600.758..600.901 rows=2414 loops=3)
                          Sort Key: nation.n_name
                          Sort Me

,n_name,revenue
0,INDONESIA,55502041.1697
1,VIETNAM,55295086.9967
2,CHINA,53724494.2566
3,INDIA,52035512.0002
4,JAPAN,45410175.6954


##### Análise da Consulta Q5

**1. Tempo Total de Execução:**
* `615.320 ms`

**2. Estratégia de Scans:**

- **Seq Scan:** Utilizadas nas tabelas menores: `region`, `nation`, `supplier` e `customer`.
- **Index Scan:** Utilizadas nas duas maiores tabelas do sistema:
    - `orders`: Usou o índice `idx_orders_custkey`.
    - `lineitem`: Usou o índice `idx_lineitem_orderkey`.

**3. Estratégia de Joins:**

- **Hash Join:** Utilizado para ligar as tabelas `region` -> `nation` -> `customer` e para a validação final com `supplier`.
- **Nested Loop:** Utilizado para conectar o resultado dos clientes aos pedidos `orders` e itens `lineitem`.

**4. Comentário da Análise:**

O banco começou filtrando `region` para encontrar 'ASIA'. Como é muito pequeno, ele usou isso para filtrar `nation` e depois `customer` rapidamente via **Hash Joins**.

Uma vez que o banco tinha a lista de clientes da Ásia, ele mudou a estratégia. Em vez de varrer a tabela gigante de `orders`, ele usou um **Nested Loop** com **Index Scan**.

- Para cada cliente asiático encontrado, ele foi no índice de pedidos e buscou pontualmente os pedidos dele.
- Em seguida, repetiu o processo para buscar os itens `lineitem` desses pedidos.

Essa combinação evitou que o banco precisasse ler as tabelas gigantes inteiras, lendo apenas os blocos de dados referentes à região 'ASIA'.

#### Consulta 6

In [39]:
df_q6 = consultas_tarefa_4(sql_q6)
df_q6

=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===
Finalize Aggregate  (cost=170006.04..170006.05 rows=1 width=32) (actual time=496.231..506.251 rows=1 loops=1)
  ->  Gather  (cost=170005.82..170006.03 rows=2 width=32) (actual time=496.079..506.220 rows=3 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Partial Aggregate  (cost=169005.82..169005.83 rows=1 width=32) (actual time=478.207..478.208 rows=1 loops=3)
              ->  Parallel Seq Scan on lineitem  (cost=0.00..168767.02 rows=47760 width=12) (actual time=9.472..462.084 rows=38053 loops=3)
                    Filter: ((l_shipdate >= '1994-01-01'::date) AND (l_shipdate < '1995-01-01 00:00:00'::timestamp without time zone) AND (l_discount >= 0.05) AND (l_discount <= 0.07) AND (l_quantity < '24'::numeric))
                    Rows Removed by Filter: 1962352
Planning Time: 0.927 ms
JIT:
  Functions: 17
  Options: Inlining false, Optimization false, Expressions true, Deforming true
  Timing: Generation 2.

,revenue
0,123141078.2283


#### Consulta 7

In [32]:
df_q7 = consultas_tarefa_4(sql_q7)
df_q7

=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===
Finalize GroupAggregate  (cost=72798.56..73625.00 rows=6115 width=272) (actual time=738.862..745.709 rows=4 loops=1)
  Group Key: n1.n_name, n2.n_name, (EXTRACT(year FROM lineitem.l_shipdate))
  ->  Gather Merge  (cost=72798.56..73469.57 rows=5096 width=272) (actual time=738.556..745.688 rows=12 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Partial GroupAggregate  (cost=71798.54..71881.35 rows=2548 width=272) (actual time=733.105..733.983 rows=4 loops=3)
              Group Key: n1.n_name, n2.n_name, (EXTRACT(year FROM lineitem.l_shipdate))
              ->  Sort  (cost=71798.54..71804.91 rows=2548 width=252) (actual time=732.743..732.907 rows=1975 loops=3)
                    Sort Key: n1.n_name, n2.n_name, (EXTRACT(year FROM lineitem.l_shipdate))
                    Sort Method: quicksort  Memory: 332kB
                    Worker 0:  Sort Method: quicksort  Memory: 328kB
                    Worker 1:  S

,supp_nation,cust_nation,l_year,revenue
0,FRANCE,GERMANY,1995,54639732.7336
1,FRANCE,GERMANY,1996,54633083.3076
2,GERMANY,FRANCE,1995,52531746.6697
3,GERMANY,FRANCE,1996,52520549.0224


#### Consulta 9

In [33]:
df_q9 = consultas_tarefa_4(sql_q9)
df_q9

=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===
Finalize GroupAggregate  (cost=76906.69..95315.42 rows=60150 width=168) (actual time=2784.939..2900.068 rows=175 loops=1)
  Group Key: nation.n_name, (EXTRACT(year FROM orders.o_orderdate))
  ->  Gather Merge  (cost=76906.69..93210.17 rows=120300 width=168) (actual time=2784.344..2899.523 rows=525 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Partial GroupAggregate  (cost=75906.67..78324.54 rows=60150 width=168) (actual time=2780.284..2887.454 rows=175 loops=3)
              Group Key: nation.n_name, (EXTRACT(year FROM orders.o_orderdate))
              ->  Sort  (cost=75906.67..76096.12 rows=75781 width=159) (actual time=2779.951..2812.565 rows=106468 loops=3)
                    Sort Key: nation.n_name, (EXTRACT(year FROM orders.o_orderdate)) DESC
                    Sort Method: external merge  Disk: 6936kB
                    Worker 0:  Sort Method: external merge  Disk: 6936kB
                    Wor

,nation,o_year,sum_profit
0,ALGERIA,1998,27136900.1803
1,ALGERIA,1997,48611833.4962
2,ALGERIA,1996,48285482.6782
3,ALGERIA,1995,44402273.5999
4,ALGERIA,1994,48694008.0668
5,ALGERIA,1993,46044207.7838
6,ALGERIA,1992,45636849.4881
7,ARGENTINA,1998,28341663.7848
8,ARGENTINA,1997,47143964.1176
9,ARGENTINA,1996,45255278.6021


#### Consulta 10

In [34]:
df_q10 = consultas_tarefa_4(sql_q10)
df_q10

=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===
Limit  (cost=166122.47..166122.52 rows=20 width=280) (actual time=579.632..590.769 rows=20 loops=1)
  ->  Sort  (cost=166122.47..166263.73 rows=56507 width=280) (actual time=556.524..567.660 rows=20 loops=1)
        Sort Key: (sum((lineitem.l_extendedprice * ('1'::numeric - lineitem.l_discount)))) DESC
        Sort Method: top-N heapsort  Memory: 33kB
        ->  Finalize GroupAggregate  (cost=157358.76..164618.84 rows=56507 width=280) (actual time=466.706..555.142 rows=37967 loops=1)
              Group Key: customer.c_custkey, nation.n_name
              ->  Gather Merge  (cost=157358.76..163441.60 rows=47090 width=280) (actual time=466.668..524.020 rows=37967 loops=1)
                    Workers Planned: 2
                    Workers Launched: 2
                    ->  Partial GroupAggregate  (cost=156358.73..157006.22 rows=23545 width=280) (actual time=438.622..469.547 rows=12656 loops=3)
                          Group Key: custo

,c_custkey,c_name,revenue,c_acctbal,n_name,c_address,c_phone,c_comment
0,57040,Customer#000057040,734235.2455,632.87,JAPAN,Eioyzjf4pp,22-895-641-3466,sits. slyly regular requests sleep alongside o...
1,143347,Customer#000143347,721002.6948,2557.47,EGYPT,"1aReFYv,Kw4",14-742-935-3718,ggle carefully enticing requests. final deposi...
2,60838,Customer#000060838,679127.3077,2454.77,BRAZIL,64EaJ5vMAHWJlBOxJklpNc2RJiWE,12-913-494-9813,need to boost against the slyly regular account
3,101998,Customer#000101998,637029.5667,3790.89,UNITED KINGDOM,01c9CILnNtfOQYmZj,33-593-865-6378,ress foxes wake slyly after the bold excuses. ...
4,125341,Customer#000125341,633508.0860,4983.51,GERMANY,S29ODD6bceU8QSuuEJznkNaK,17-582-695-5962,arefully even depths. blithely even excuses sl...
5,25501,Customer#000025501,620269.7849,7725.04,ETHIOPIA,"W556MXuoiaYCCZamJI,Rn0B4ACUGdkQ8DZ",15-874-808-6793,he pending instructions wake carefully at the ...
6,115831,Customer#000115831,596423.8672,5098.10,FRANCE,rFeBbEEyk dl ne7zV5fDrmiq1oK09wV7pxqCgIc,16-715-386-3788,l somas sleep. furiously final deposits wake b...
7,84223,Customer#000084223,594998.0239,528.65,UNITED KINGDOM,nAVZCs6BaWap rrM27N 2qBnzc5WBauxbA,33-442-824-8191,"slyly final deposits haggle regular, pending ..."
8,54289,Customer#000054289,585603.3918,5583.02,IRAN,"vXCxoCsU0Bad5JQI ,oobkZ",20-834-292-4707,ely special foxes are quickly finally ironic p
9,39922,Customer#000039922,584878.1134,7321.11,GERMANY,Zgy4s50l2GKN4pLDPBU8m342gIw6R,17-147-757-8036,y final requests. furiously final foxes cajole...


#### Consulta 12

In [35]:
df_q12 = consultas_tarefa_4(sql_q12)
df_q12

=== PLANO DE EXECUÇÃO E TEMPO (EXPLAIN ANALYZE) ===
Finalize GroupAggregate  (cost=209395.16..209641.40 rows=7 width=27) (actual time=805.000..815.977 rows=2 loops=1)
  Group Key: lineitem.l_shipmode
  ->  Gather Merge  (cost=209395.16..209641.23 rows=14 width=27) (actual time=801.937..815.877 rows=6 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Partial GroupAggregate  (cost=208395.14..208639.59 rows=7 width=27) (actual time=781.489..783.633 rows=2 loops=3)
              Group Key: lineitem.l_shipmode
              ->  Sort  (cost=208395.14..208425.69 rows=12219 width=27) (actual time=779.270..780.040 rows=10329 loops=3)
                    Sort Key: lineitem.l_shipmode
                    Sort Method: quicksort  Memory: 1154kB
                    Worker 0:  Sort Method: quicksort  Memory: 1216kB
                    Worker 1:  Sort Method: quicksort  Memory: 1204kB
                    ->  Parallel Hash Join  (cost=168919.75..207565.66 rows=12219 width=27) 

,l_shipmode,high_line_count,low_line_count
0,MAIL,6202,9324
1,SHIP,6200,9262


# Parte 2

O objetivo desta parte do trabalho é analisar o comportamento dos índices das tabelas do SGBD através do exame e análise das tabelas de estatísticas para consultas SQL sobre uma tabela criada com dados aleatórios.

# Parte 3

O objetivo desta parte do trabalho é estudar o comportamento dos otimizadores de consulta dos SGBDs através do exame e análise dos planos de execução para consultas SQL sobre tabelas que serão fornecidos. Será bastante utilizado o comando EXPLAIN ANALYZE, que permite visualizar todas as etapas envolvidas no processamento de uma consulta. Usaremos para isso a tabela [“movies”](https://drive.google.com/file/d/1W6wovSsVu4B0OIo_tsSBBHi8WRKQqnat/view?usp=drive_link).

# Parte 4

O objetivo desta parte do trabalho é experimentar estratégias para utilização de transações e níveis de isolamento em SGBDs relacionais. As tarefas envolvem uma simulação de um sistema de reservas de passagem áreas.